## [12-27] 流平衡写的是有问题的

## [12-30] 输出结果是遍历所有点不重复边的

In [1]:
import gurobipy as gb
from gurobipy import*
import openpyxl  
import numpy as np
import math
import time
import copy
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import heapq
import os
import sys


In [2]:
# PARA
M = 10E5
ME = 0.001
ISSme = 1

In [3]:
def get_values(filename,id = 0,delme = 0):
    wb = openpyxl.load_workbook(filename)
    sheet_names = wb.sheetnames
    print(sheet_names)

    sheet = wb[sheet_names[id]]
    maxrow = sheet.max_row 
    maxcol = sheet.max_column 
    value = sheet.cell(row=2, column=1).value
    alldata = []

    for row in sheet.iter_rows(min_row=1, max_row=maxrow, min_col=1, max_col=maxcol): # type: ignore
        row_data = [cell.value for cell in row]
        alldata.append(row_data)
    station_name = alldata[0][1:]

    if delme:
        del alldata[0]
        for i in range(len(alldata)):
            del alldata[i][0]


    return alldata,station_name

In [4]:
def refineme(data):
    for i in range(len(data)):
        for j in range(len(data[i])):
            if data[i][j] is None:
                data[i][j] = M
    return data

In [5]:
def convme(data,title):
    degree = {}
    alldegree = 0
    for i,row in enumerate(data):
        
        degree[title[i]] = sum(row)
        alldegree = alldegree + sum(row)
    return degree,alldegree


In [6]:
def findme_maxdegreee(degree):
    max_degree = max(degree.values())

    max_degree_node = []
    for key in degree:
        if int(degree[key]) == int(max_degree):
            max_degree_node.append(key)
    return(max_degree,max_degree_node)

In [7]:
def picme(station_name,MyOD):
    plt.figure(figsize=(10, 6))
    MyOD = np.array(MyOD)
    plt.imshow(MyOD, cmap='Blues', aspect='auto', interpolation='nearest')
    plt.colorbar(label='Value')
    plt.xticks(ticks=np.arange(len(station_name)), labels=station_name, rotation=45)
    plt.yticks(ticks=np.arange(MyOD.shape[0]), labels=[f'Row {i+1}' for i in range(MyOD.shape[0])])
    plt.title('Heatmap of Matrix')
    plt.xlabel('Columns')
    plt.ylabel('Rows')
    plt.tight_layout()
    plt.show()

In [8]:
MyOD,station_name = get_values('OD.xlsx',delme=1)

['Sheet1']


In [9]:
MyOD = refineme(MyOD)

In [10]:
station_name.insert(0,'d')
station_name.append('a')

In [11]:
station_name

['d',
 'V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'a']

In [12]:
# 设置插入的值
value_to_insert = M/20

# 示例数据


# 插入新列（在每行的开头和结尾都插入100）
for row in MyOD:
    row.insert(0, value_to_insert)  # 插入开头的列
    row.append(value_to_insert)     # 插入结尾的列

# 打印结果
print("After inserting columns:")
for row in MyOD:
    print(row)

print('\n')

# 插入新行（在开头和结尾插入100的行）
new_row = [value_to_insert for _ in range(len(MyOD[0]))]  # 创建一个独立的新行
MyOD.insert(0, new_row)  # 在开头插入新行
MyOD.append([value_to_insert for _ in range(len(MyOD[0]))])  # 在结尾插入独立的新行

# 打印结果
print("After inserting rows:")
for row in MyOD:
    print(row)


After inserting columns:
[50000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 50000.0]
[50000.0, 1, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 50000.0]
[50000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 50000.0]
[50000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 50000.0]
[50000.0, 1000000.0, 1, 1000000.0, 1000000.0, 1000000.0, 1, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 50000.0]
[50000.0, 1000000.0, 100

In [13]:
MyOD == copy.deepcopy(MyOD)

True

In [14]:
for i, row in enumerate(MyOD):
    for j, item in enumerate(row):
        # 第一行第一个元素
        if i == 0 and j == 0:

            MyOD[i][j] = M

        # 第一行最后一个元素
        elif i == 0 and j == len(row) - 1:
            MyOD[i][j] = M

        # 第一列或最后一列的元素
        elif j == 0 :
            MyOD[i][j] = M

        # 最后一行的元素
        elif i == len(MyOD) - 1:

            MyOD[i][j] = M


In [15]:
# 打印结果
for row in MyOD:
    print(row)

[1000000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 50000.0, 1000000.0]
[1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 50000.0]
[1000000.0, 1, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 50000.0]
[1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 50000.0]
[1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 1000000.0, 50000.0]
[1000000.0, 1000000.0, 1, 1000000.0, 1000000.0, 10000

In [16]:
model1 = gb.Model()

In [17]:
x = {}
y = {}
z = {}
for i in range(len(MyOD)):
    for j in range(len(MyOD)):
        if i!= j:
            x[i,j] = model1.addVar(vtype=gb.GRB.INTEGER,lb = 0,name = f'[x{station_name[i]},{station_name[j]}]')
            y[i,j] = model1.addVar(vtype=gb.GRB.BINARY,name = f'[y{station_name[i]},{station_name[j]}]')
            z[i,j] = model1.addVar(vtype=gb.GRB.BINARY,name = f'[z{station_name[i]},{station_name[j]}]')

In [18]:
for i in range(0, len(MyOD)-1, 1):
    for j in range(0, len(MyOD)-1, 1):
        if i != j:
            # 如果 x[i,j] >= 1, 则 z[i,j] = 1
            model1.addConstr(x[i,j] - 1 <= M * (z[i,j] - 1))
            
            # 如果 x[i,j] = 0, 则 z[i,j] = 0
            model1.addConstr(x[i,j] <= M * z[i,j])



In [19]:
'''for i in range(0,len(MyOD)-1,1 ):
    for j in range(0,len(MyOD)-1,1 ):
        if i != j:
            model1.addConstr(x[i,j] >= M*(z[i,j]-1)) # 1+ -> 1

            model1.addConstr(x[i,j] <= ME*z[i,j] )'''

'for i in range(0,len(MyOD)-1,1 ):\n    for j in range(0,len(MyOD)-1,1 ):\n        if i != j:\n            model1.addConstr(x[i,j] >= M*(z[i,j]-1)) # 1+ -> 1\n\n            model1.addConstr(x[i,j] <= ME*z[i,j] )'

In [20]:
# flow consit
model1.addConstr(quicksum(x[0,i] for i in range(1,len(MyOD)-1,1 ))== 
                 quicksum(x[j,len(MyOD)-1]  for j in range(1,len(MyOD)-1,1) ))

<gurobi.Constr *Awaiting Model Update*>

In [21]:
for j in range(1,len(MyOD)-1,1):
    model1.addConstr(quicksum(x[i,j] for i in range(len(MyOD)) if i!= j) 
                     == quicksum(x[j,k] for k in range(len(MyOD) )if k!= j))

In [22]:
# node must visit
model1.addConstr(quicksum(x[0,i] for i in range(1,len(MyOD)-1,1) ) >= 1)

<gurobi.Constr *Awaiting Model Update*>

In [23]:
model1.addConstr(quicksum(x[k,len(MyOD)-1] for k in range(1,len(MyOD)-1,1) ) >= 1)

<gurobi.Constr *Awaiting Model Update*>

In [24]:
for i in range(1,len(MyOD)-1,1 ):

    model1.addConstr(quicksum(x[k,i] for k in range(len(MyOD)) if k !=i) >= 1)


In [25]:
# path one direction
for i in range(0,len(MyOD)-1,1 ):
    for j in range(0,len(MyOD)-1,1 ):
        if i != j:
            model1.addConstr(x[i,j]<= M*y[i,j])
            model1.addConstr(x[j,i]<=M*(1-y[i,j]))
        

In [26]:
Z1 = quicksum(int(MyOD[i][j])*x[i,j] for i in range(len(MyOD)) for j in range(len(MyOD))if i!= j)

In [27]:
#main
Z2 = quicksum(int(MyOD[0][j])*z[0,j]  for j in range(len(MyOD))if j != 0) + quicksum(int(MyOD[j][len(MyOD)-1])*z[j,len(MyOD)-1]  for j in range(len(MyOD)) if j != len(MyOD)-1) + quicksum(int(MyOD[i][j])*x[i,j] for i in range(1,len(MyOD)-1,1 )for j in range(len(MyOD))if i != j)


In [28]:
Z3 = quicksum(int(MyOD[i][j])*z[i,j] for i in range(len(MyOD)) for j in range(len(MyOD))if i!= j)

In [29]:
model1.setObjective(Z2, sense=gb.GRB.MINIMIZE)

In [30]:
model1.update()

In [31]:
model1.write("model.lp")

In [32]:
print('\n')
model1.setParam(gb.GRB.Param.MIPGap, 0.005)
model1.setParam(gb.GRB.Param.TimeLimit, 3600*2)
model1.optimize()

try:
    if model1.ObjVal >= M*2:
        print(f'Model result of {model1.ObjVal:.2f}  exceed M:{M:.2f}')
        print('Result may out of node\\arc ability ')
    else:
        print(model1.ObjVal)
except Exception as e:
    print(e)
    print('\033[91m\nFatal! Model is INFEASIBLE.')

    if ISSme:
        model1.computeIIS()
        model1.write("modelv0.3.ilp")
    else:
        print('\033[5m\n ISS CHECK CANCEL \n\033[0m')



Set parameter MIPGap to value 0.005
Set parameter TimeLimit to value 7200
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-14600KF, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 1123 rows, 918 columns and 3056 nonzeros
Model fingerprint: 0xed930a2e
Variable types: 0 continuous, 918 integer (612 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+00, 2e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 952 rows and 614 columns
Presolve time: 0.00s
Presolved: 171 rows, 304 columns, 1152 nonzeros
Variable types: 0 continuous, 304 integer (272 binary)
Found heuristic solution: objective 1.585000e+07
Found heuristic solution: objective 1.385000e+07

Root relaxation: objective 1.050011e+06, 56 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current N

In [33]:
#print(p[reflector[i], reflector[j],k].varName,p[reflector[i], reflector[j],k].x)

roadrec = []
mynode = [0,0]

for i in range(len(MyOD)):
    for j in range(len(MyOD)):
        if i!= j:
            if x[i,j].x >= 0.001 :
                if MyOD[i][j] >= M:
                    print('OD Shhould not choose ',x[i,j].varName,x[i,j].x)
                else:
                    print(x[i,j].varName,x[i,j].x,x[i,j].x*MyOD[i][j])
                    if i == 0:
                        mynode[0] = mynode[0]+1
                    if j == len(MyOD)-1:
                        mynode[1] = mynode[1]+1

                    roadrec.append([station_name[i],station_name[j],x[i,j].x,x[i,j].x*MyOD[i][j]])

[xd,V2] 1.0 50000.0
[xd,V4] 1.0 50000.0
[xd,V8] 1.0 50000.0
[xd,V15] 1.0 50000.0
[xd,V16] 1.0 50000.0
[xV1,a] 1.0 50000.0
[xV2,V1] 1.0 1.0
[xV3,a] 1.0 50000.0
[xV4,a] 1.0 50000.0
[xV5,V6] 1.0 1.0
[xV6,V7] 1.0 1.0
[xV7,V9] 1.0 1.0
[xV8,V5] 1.0 1.0
[xV9,V10] 1.0 1.0
[xV10,V13] 1.0 1.0
[xV11,a] 1.0 50000.0
[xV12,V11] 1.0 1.0
[xV13,V14] 1.0 1.0
[xV14,a] 1.0 50000.0
[xV15,V3] 1.0 1.0
[xV16,V12] 1.0 1.0


In [34]:

x[0,2].x

1.0

In [35]:
z[0,2].x

1.0

In [36]:
# 构建图的邻接表
graph = {}

for entry in roadrec:
    start, end, value1, value2 = entry
    if start not in graph:
        graph[start] = []
    graph[start].append(end)

# 输出邻接表检查是否正确构建
print("Graph adjacency list:")
for node, neighbors in graph.items():
    print(f"{node}: {neighbors}")

# 深度优先搜索（DFS）查找从 'xd' 到 'a' 的路径
def dfs(graph, start, end, path):
    path.append(start)
    print(f"Visiting: {start}")  # 调试：输出访问的节点
    
    if start == end:
        return path
    
    if start in graph:
        for neighbor in graph[start]:
            if neighbor not in path:  # 防止回环
                result = dfs(graph, neighbor, end, path.copy())
                if result:
                    return result
    return None

# 从 'd' 到 'a' 查找路径
start_node = "d"
end_node = "a"
path = dfs(graph, start_node, end_node, [])

# 输出路径
if path:
    print("Path found:", " -> ".join(path))
else:
    print("No path found")

Graph adjacency list:
d: ['V2', 'V4', 'V8', 'V15', 'V16']
V1: ['a']
V2: ['V1']
V3: ['a']
V4: ['a']
V5: ['V6']
V6: ['V7']
V7: ['V9']
V8: ['V5']
V9: ['V10']
V10: ['V13']
V11: ['a']
V12: ['V11']
V13: ['V14']
V14: ['a']
V15: ['V3']
V16: ['V12']
Visiting: d
Visiting: V2
Visiting: V1
Visiting: a
Path found: d -> V2 -> V1 -> a
